In [1]:
import psycopg2
import pandas as pd
import numpy as np
import polars as pl

import matplotlib.pyplot as plt
import seaborn as sns

import csv

import warnings

warnings.filterwarnings("ignore")
sns.set_style()
plt.rcParams["figure.figsize"] = (12, 6)


# Parte 1

In [2]:
conn = psycopg2.connect("dbname=javier_rojas user=postgres password=2719")
cursor = conn.cursor()

In [3]:
df_train = pd.read_csv("input/train_cupid.csv")
df_train.columns = df_train.columns.str.replace(' ', '')
df_test = pd.read_csv("input/test_cupid.csv")

In [4]:
query_train = """
CREATE TABLE train_cupid(
 age INTEGER,
 height INTEGER,
 virgo INTEGER,
 taurus INTEGER,
 scorpio INTEGER,
 pisces INTEGER,
 libra INTEGER,
 leo INTEGER,
 gemini INTEGER,
 aries INTEGER,
 aquarius INTEGER,
 cancer INTEGER,
 sagittarius INTEGER,
 asian INTEGER,
 hispaniclatin INTEGER,
 black INTEGER,
 indian INTEGER,
 pacificislander INTEGER,
 nativeamerican INTEGER,
 middleeastern INTEGER,
 colorado INTEGER,
 newyork INTEGER,
 oregon INTEGER,
 arizona INTEGER,
 hawaii INTEGER,
 montana INTEGER,
 wisconsin INTEGER,
 virginia INTEGER,
 spain INTEGER,
 nevada INTEGER,
 illinois INTEGER,
 vietnam INTEGER,
 ireland INTEGER,
 louisiana INTEGER,
 michigan INTEGER,
 texas INTEGER,
 unitedkingdom INTEGER,
 massachusetts INTEGER,
 northcarolina INTEGER,
 idaho INTEGER,
 mississippi INTEGER,
 newjersey INTEGER,
 florida INTEGER,
 minnesota INTEGER,
 georgia INTEGER,
 utah INTEGER,
 washington INTEGER,
 westvirginia INTEGER,
 connecticut INTEGER,
 tennessee INTEGER,
 rhodeisland INTEGER,
 districtofcolumbia INTEGER,
 canada INTEGER,
 missouri INTEGER,
 germany INTEGER,
 pennsylvania INTEGER,
 netherlands INTEGER,
 switzerland INTEGER,
 mexico INTEGER,
 ohio INTEGER,
 agnosticism INTEGER,
 atheism INTEGER,
 catholicism INTEGER,
 buddhism INTEGER,
 judaism INTEGER,
 hinduism INTEGER,
 islam INTEGER,
 pro_dogs INTEGER,
 pro_cats INTEGER,
 spanish INTEGER,
 chinese INTEGER,
 french INTEGER,
 german INTEGER,
 single INTEGER,
 seeing_someone INTEGER,
 available INTEGER,
 employed INTEGER,
 income_between_25_50 INTEGER,
 income_between_50_75 INTEGER,
 income_over_75 INTEGER,
 drugs_often INTEGER,
 drugs_sometimes INTEGER,
 drinks_notatall INTEGER,
 drinks_often INTEGER,
 drinks_rarely INTEGER,
 drinks_socially INTEGER,
 drinks_veryoften INTEGER,
 orientation_gay INTEGER,
 orientation_straight INTEGER,
 sex_m INTEGER,
 smokes_sometimes INTEGER,
 smokes_tryingtoquit INTEGER,
 smokes_whendrinking INTEGER,
 smokes_yes INTEGER,
 body_type_overweight INTEGER,
 body_type_regular INTEGER,
 education_high_school INTEGER,
 education_undergrad_university INTEGER
);
"""

query_test = """
CREATE TABLE test_cupid(
 age INTEGER,
 height INTEGER,
 virgo INTEGER,
 taurus INTEGER,
 scorpio INTEGER,
 pisces INTEGER,
 libra INTEGER,
 leo INTEGER,
 gemini INTEGER,
 aries INTEGER,
 aquarius INTEGER,
 cancer INTEGER,
 sagittarius INTEGER,
 asian INTEGER,
 hispaniclatin INTEGER,
 black INTEGER,
 indian INTEGER,
 pacificislander INTEGER,
 nativeamerican INTEGER,
 middleeastern INTEGER,
 colorado INTEGER,
 newyork INTEGER,
 oregon INTEGER,
 arizona INTEGER,
 hawaii INTEGER,
 montana INTEGER,
 wisconsin INTEGER,
 virginia INTEGER,
 spain INTEGER,
 nevada INTEGER,
 illinois INTEGER,
 vietnam INTEGER,
 ireland INTEGER,
 louisiana INTEGER,
 michigan INTEGER,
 texas INTEGER,
 unitedkingdom INTEGER,
 massachusetts INTEGER,
 northcarolina INTEGER,
 idaho INTEGER,
 mississippi INTEGER,
 newjersey INTEGER,
 florida INTEGER,
 minnesota INTEGER,
 georgia INTEGER,
 utah INTEGER,
 washington INTEGER,
 westvirginia INTEGER,
 connecticut INTEGER,
 tennessee INTEGER,
 rhodeisland INTEGER,
 districtofcolumbia INTEGER,
 canada INTEGER,
 missouri INTEGER,
 germany INTEGER,
 pennsylvania INTEGER,
 netherlands INTEGER,
 switzerland INTEGER,
 mexico INTEGER,
 ohio INTEGER,
 agnosticism INTEGER,
 atheism INTEGER,
 catholicism INTEGER,
 buddhism INTEGER,
 judaism INTEGER,
 hinduism INTEGER,
 islam INTEGER,
 pro_dogs INTEGER,
 pro_cats INTEGER,
 spanish INTEGER,
 chinese INTEGER,
 french INTEGER,
 german INTEGER,
 single INTEGER,
 seeing_someone INTEGER,
 available INTEGER,
 employed INTEGER,
 income_between_25_50 INTEGER,
 income_between_50_75 INTEGER,
 income_over_75 INTEGER,
 drugs_often INTEGER,
 drugs_sometimes INTEGER,
 drinks_notatall INTEGER,
 drinks_often INTEGER,
 drinks_rarely INTEGER,
 drinks_socially INTEGER,
 drinks_veryoften INTEGER,
 orientation_gay INTEGER,
 orientation_straight INTEGER,
 sex_m INTEGER,
 smokes_sometimes INTEGER,
 smokes_tryingtoquit INTEGER,
 smokes_whendrinking INTEGER,
 smokes_yes INTEGER,
 body_type_overweight INTEGER,
 body_type_regular INTEGER,
 education_high_school INTEGER,
 education_undergrad_university INTEGER
);
"""

cursor.execute(query_train)
cursor.execute(query_test)

conn.commit()

In [5]:
with open('input/train_cupid.csv') as file:
    reader = csv.reader(file)
    next(reader)

    for row in reader:
        cursor.execute("INSERT INTO data_titanic")

SyntaxError: syntax error at end of input
LINE 1: INSERT INTO data_titanic
                                ^
